In [1]:
import os
import diff_classifier.imagej as ij
import boto3
import os.path as op
import diff_classifier.aws as aws
import cloudknot as ck
import diff_classifier.knotlets as kn
import numpy as np

In [3]:
rows = ['0', '1', '2', '3']
folder = '01_18_Experiment'

missing = []
for i in range(10, 15):
    for row in rows:
        for col in rows:
            missing.append("{}/P1/P1_S2_R_00{}_{}_{}.tif".format(folder, i, row, col))

for i in range(10, 15):
    for row in rows:
        for col in rows:
            missing.append("{}/P2/P2_S3_L_00{}_{}_{}.tif".format(folder, i, row, col))
    
for i in range(0, 15):
    for row in rows:
        for col in rows:
            missing.append("{}/P3/P3_S3_L_{}_{}_{}.tif".format(folder, "%04d" % i, row, col))

In [4]:
to_track = {}
knot = {}
result_futures = {}
start_knot = 30

pups = ["P1", "P2", "P3"]
slices = ["S1", "S2", "S3"]
folder = '01_18_Experiment'
start_vid = [0, 3, 6, 9, 12]
end_vid = [3, 6, 9, 12, 15]

hemis = ["R", "L"]
rows = ['0', '1', '2', '3']

github_installs=('https://github.com/ccurtis7/diff_classifier.git@scaleup')
my_image = ck.DockerImage(func=kn.download_and_track, base_image='arokem/python3-fiji:0.3', github_installs=github_installs)

docker_file = open(my_image.docker_path)
docker_string = docker_file.read()
docker_file.close()

cut1 = docker_string.find('# Create a default user')
cut2 = docker_string[cut1:].find('\n\n') + cut1
new_docker = docker_string[0:cut1]+docker_string[cut2:]
docker_overwrite = open(my_image.docker_path, 'w')
docker_overwrite.write(new_docker)
docker_overwrite.close()

for pup in pups:
    for slic in slices:
        for start_ind in range(0, 5):

            run_name = '{}_{}_{}'.format(pup, slic, start_ind)
            to_track[run_name] = []
            for hemi in hemis:
                for i in range(start_vid[start_ind], end_vid[start_ind]):
                    for row in rows:
                        for col in rows:
                            name = "{}/{}/{}_{}_{}_{}_{}_{}.tif".format(folder, pup, pup, slic, hemi, "%04d" % i, row, col)
                            if not name in missing:
                                to_track[run_name].append(name)

            test_length = len(to_track[run_name])
            print('Number of nodes to be loaded: {}'.format(test_length))
            
            knot[run_name] = ck.Knot(name='download_and_track_with_my_image_{}'.format(start_knot),
                           docker_image = my_image,
                           pars_policies=('AmazonS3FullAccess',))
            result_futures[run_name] = knot[run_name].map(to_track[run_name])
            start_knot = start_knot + 1
            print('Next knot name: {}'.format(start_knot))

Number of nodes to be loaded: 96
Next knot name: 31
Number of nodes to be loaded: 96
Next knot name: 32
Number of nodes to be loaded: 96
Next knot name: 33
Number of nodes to be loaded: 96
Next knot name: 34
Number of nodes to be loaded: 96
Next knot name: 35
Number of nodes to be loaded: 96
Next knot name: 36
Number of nodes to be loaded: 96


KeyboardInterrupt: 

In [ ]:
# github_installs=('https://github.com/ccurtis7/diff_classifier.git@scaleup')
# my_image = ck.DockerImage(func=kn.download_and_track, base_image='arokem/python3-fiji:0.3', github_installs=github_installs)

# docker_file = open(my_image.docker_path)
# docker_string = docker_file.read()
# docker_file.close()

# cut1 = docker_string.find('# Create a default user')
# cut2 = docker_string[cut1:].find('\n\n') + cut1
# new_docker = docker_string[0:cut1]+docker_string[cut2:]
# docker_overwrite = open(my_image.docker_path, 'w')
# docker_overwrite.write(new_docker)
# docker_overwrite.close()

# knot1 = ck.Knot(name='download_and_track_with_my_image_27',
#                docker_image = my_image,
#                pars_policies=('AmazonS3FullAccess',))
# result_futures = knot1.map(to_track)

In [ ]:
knot1.view_jobs()

In [ ]:
knot1.clobber()

In [5]:
for key in knot:
    print key
    knot[key].clobber()

P1_S2_1
P1_S2_0
P1_S1_4
P1_S1_2
P1_S1_3
P1_S1_0
P1_S1_1


In [ ]:
for key in knot